In [2]:
#TEST


import pandas as pd
import os
import sys
import json #reading java strings into python dictionaries
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import argparse
import re
import json



In [3]:
#FOR ZOONIVERSE DATA

pd.set_option('display.max_rows', None)
file_path = r"C:\Users\xomad\ICECUBE\data\output_retirement_lim15\name-that-neutrino-classifications.csv"
subj_ids=[]
filenames = []
runs = []
events = []
ntn_subjects = pd.read_csv(file_path)
for i in range(len(ntn_subjects)):
    dict1 = json.loads(ntn_subjects['subject_data'][i]) #each event/subject id is its own dictionary in classifications csv. 

    for key, value in dict1.items():  # Iterate through each subject ID and its associated data
        if key in subj_ids:  # Skip if subject_id is already added
            continue
        subj_ids.append(key)  #add the event (key) to a list
        filename = value['Filename']
        match = re.match(r'run_(\d+)_event_(\d+)_', filename)
        
        if match:
            run_number = match.group(1)  # Extract the run number
            event_number = match.group(2)  # Extract the event number
        else:
            run_number = None
            event_number = None
            print(f"Warning: Filename {filename} does not match the expected pattern.")

         # Append extracted values to their respective lists
        filenames.append(filename)
        runs.append(run_number)
        events.append(event_number)

# # Combine the collected data into a DataFrame for easier handling SAVE TO NEW FILE INSTEAD
print(len(subj_ids))
df = pd.DataFrame({ 'subject_id': subj_ids, 'filename': filenames, 'run': runs, 'event': events})

# Save the DataFrame to a new CSV file
df.to_csv(r'..\data\subject_data.csv', index=False)

print(f"Data saved to 'subject_data.csv' with {len(df)} records.")



# FOR i3 DATA
all_sim_DNN_data = []
events_len = 0


MP_i3 = [f for f in os.listdir(r"..\MC i3 data")]  

for file in MP_i3:
    sim_DNN_data = pd.read_excel(os.path.join(r"..\MC i3 data", file))

    #check how many events
    eventsperfile = int(len(sim_DNN_data))
    events_len += eventsperfile

    all_sim_DNN_data.append(sim_DNN_data)

# Concatenate all the dataframes in the list into a single dataframe
combined_data = pd.concat(all_sim_DNN_data, ignore_index=True)

# Save the combined data to a new CSV file
combined_data.to_csv(r'..\data\combined_sim_DNN_data.csv', index=False)



7840
Data saved to 'subject_data.csv' with 7840 records.


In [7]:
#matching dnn/sim to user data, removing all duplicates 

file_path_zoo = r"C:\Users\xomad\ICECUBE\data\subject_data.csv"
file_path_i3 = r"C:\Users\xomad\ICECUBE\data\combined_sim_DNN_data.csv"

zoodata = pd.read_csv(file_path_zoo)
i3data = pd.read_csv(file_path_i3)

# Ensure 'run' and 'event' columns are treated as strings for accurate comparison
zoodata['run'] = zoodata['run'].astype(str)
zoodata['event'] = zoodata['event'].astype(str)

i3data['run'] = i3data['run'].astype(int).astype(str)  # Remove decimals from i3_run
i3data['event'] = i3data['event'].astype(int).astype(str)

# Create unique keys for 'zoo_run_event_key' and 'i3_run_event_key'
zoodata['zoo_run_event_key'] = zoodata['run'].astype(str) + "_" + zoodata['event'].astype(str)
i3data['i3_run_event_key'] = i3data['run'].astype(str) + "_" + i3data['event'].astype(str)

# Merge datasets on the event keys
matched_data = pd.merge(zoodata, i3data, left_on='zoo_run_event_key', right_on='i3_run_event_key', how='inner', suffixes=('_zoo', '_i3'))

# Clean the matched data by removing unwanted columns
matched_data_cleaned = matched_data.loc[:, ~matched_data.columns.str.startswith('Unnamed')]

# Find duplicate rows based on 'run_zoo' and 'event_zoo' columns and remove both duplicates
duplicate_keys = matched_data_cleaned.duplicated(subset=['run_zoo', 'event_zoo'], keep=False) # keep = 'first' to keep 1
matched_data_cleaned_unique = matched_data_cleaned[~duplicate_keys]

# Rename columns 'run_zoo' and 'event_zoo' to 'run' and 'event'
matched_data_cleaned_unique = matched_data_cleaned_unique.rename(columns={'run_zoo': 'run', 'event_zoo': 'event'})

# Check columns
print("Matched Data Columns:")
print(matched_data_cleaned.columns)

# Save the matched data to a new CSV
matched_data_cleaned_unique.to_csv(r'..\data\matched_sim_data.csv', index=False)


Matched Data Columns:
Index(['subject_id', 'filename', 'run_zoo', 'event_zoo', 'origidx',
       'zoo_run_event_key', 'run_i3', 'event_i3', 'truth_classification',
       'pred_skim', 'pred_cascade', 'pred_tgtrack', 'pred_starttrack',
       'pred_stoptrack', 'energy', 'zenith', 'oneweight', 'signal_charge',
       'bg_charge', 'qtot', 'qratio', 'log10_max_charge',
       '#truth_classification_label', 'max_score_val', 'idx_max_score',
       'ntn_category', 'i3_run_event_key'],
      dtype='object')


In [8]:
## 1/6 used to only exclude files with (1)

import pandas as pd
import json
import re

# Set pandas option to display all rows
pd.set_option('display.max_rows', None)

file_path = r"C:\Users\xomad\ICECUBE\data\output_retirement_lim15\name-that-neutrino-classifications.csv"
subj_ids = []
filenames = []
runs = []
events = []
origidxs = []

# Read the CSV file
ntn_subjects = pd.read_csv(file_path)

# Iterate through each subject in the CSV file
for i in range(len(ntn_subjects)):
    # Load the subject_data (which is a JSON string in the 'subject_data' column)
    dict1 = json.loads(ntn_subjects['subject_data'][i]) 

    # Iterate through each subject_id and its associated data
    for key, value in dict1.items():
        # Extract the filename and match it to run, event, and origidx using regex
        filename = value['Filename']
        
        # Skip filenames with '(1)' in the name (duplicate versions)
        if "(1)" in filename:
            continue
        
        # Use regex to extract run, event, and origidx values from the filename
        match = re.match(r'run_(\d+)_event_(\d+)_origidx_(\d+)', filename)
        
        if match:
            run_number = match.group(1)  # Extract the run number
            event_number = match.group(2)  # Extract the event number
            origidx_number = match.group(3)  # Extract the origidx number
        else:
            run_number = None
            event_number = None
            origidx_number = None
            print(f"Warning: Filename {filename} does not match the expected pattern.")

        # Append the subject_id, filename, run, event, and origidx to their respective lists
        subj_ids.append(key)  # Append subject_id (key)
        filenames.append(filename)
        runs.append(run_number)
        events.append(event_number)
        origidxs.append(origidx_number)

# Create a DataFrame with the collected data
df = pd.DataFrame({
    'subject_id': subj_ids, 
    'filename': filenames, 
    'run': runs, 
    'event': events, 
    'origidx': origidxs
})

# Remove duplicates based on the 'run', 'event', and 'origidx' columns
df_unique = df.drop_duplicates(subset=['run', 'event', 'origidx'])

# Save the DataFrame to a new CSV file
df_unique.to_csv(r'..\data\subject_data.csv', index=False)

print(f"Data saved to 'subject_data.csv' with {len(df_unique)} unique records.")



Data saved to 'subject_data.csv' with 7595 unique records.


In [7]:
#combining all MC i3 files into one
all_sim_DNN_data = []
events_len = 0


MP_i3 = [f for f in os.listdir(r"..\MC i3 data")]  

for file in MP_i3:
    sim_DNN_data = pd.read_excel(os.path.join(r"..\MC i3 data", file))

    #check how many events
    eventsperfile = int(len(sim_DNN_data))
    events_len += eventsperfile

    all_sim_DNN_data.append(sim_DNN_data)

# Concatenate all the dataframes in the list into a single dataframe
combined_data = pd.concat(all_sim_DNN_data, ignore_index=True)

# Save the combined data to a new CSV file
combined_data.to_csv(r'..\data\combined_sim_DNN_data.csv', index=False)